# Small local agents

Experimenting with agents does not require access to external services, expensive hardware, or even the cloud.
This short tutorial notebook is a introduction to language model-based agents using small language models, and demonstrating their use to query a database.

This is a repository on Github:
https://github.com/lgautier/local-small-language-agent

# Setup

## Local Ollama server

The language models will we use will be served by Ollama.
The default instructions use `podman` to run that Ollama server in a rootless container.

**If podman cannot be available, or using an isolated system where running ollama outside of
a container is not an issue, alternative instructions follow.**

This assumes that `podman` is installed. If not yet the case, the instructions are here:
https://podman.io/docs/installation

In [1]:
! podman run --replace -d --rm -p 11434:11434 -v dot_ollama:/root/.ollama --name ollama docker.io/ollama/ollama

ed86c9e2787467f1ee8c4c1e8996388bd48bbc189d8792e8e6be7800d06ddccc


If on a debian-based system where sudo-ing an installation of ollama is possible,
for example on Google Colab, the following instructions can be used. Create a new
jupyter cell with the code below, or replace the podman cell above with it.

```bash
# Derived from the the Google Cloud doc:
# https://medium.com/google-cloud/gemma-3-ollama-on-colab-a-developers-quickstart-7bbf93ab8fef
! echo $(tput setaf 1)"Installing APT packages."$(tput setaf 0)
! sudo apt-get -qq update && sudo apt-get -qq install -y pciutils lshw
! echo $(tput setaf 1)"Fetching Ollama."$(tput setaf 0)
! curl -fsSL https://ollama.com/install.sh | sh
! echo $(tput setaf 1)"Starting Ollama server."$(tput setaf 0)
! nohup ollama serve > ollama.log 2>&1 &
```

## Required Python packages

The Python packages `ollama` and `langchain`, and `langchain_ollama` are required. Make sure they are installed before continuing.

In [2]:
import dataclasses
import langchain
import langchain_ollama
import ollama

## Utility code

In order to minimize code duplication, we encapsulate most of Ollama model management into a class.

In [3]:
@dataclasses.dataclass(frozen=True)
class OllamaModel:
    """Wrapper class for models with Ollama and LangChain."""

    name: str

    def __str__(self) -> str:
        return self.name

    def has_tools(self) -> bool:
        return 'tools' in self.show().capabilities

    def is_pulled(self) -> bool:
        for mdl in ollama.list().models:
            if mdl.model == self.name:
                return True
        return False

    def pull(self):
        """Pull the model to make it available to the local Ollama instance."""
        ollama.pull(self.name)

    def show(self) -> ollama._types.ShowResponse:
        return ollama.show(self.name)

    def new_chat(self, temperature:float = 0) -> langchain_ollama.ChatOllama:
        res = langchain_ollama.ChatOllama(
            model=str(self),
            temperature=temperature
        )
        return res

### Object display customization
We also create a convenience custom HTML render for AI responses.

In [4]:
import jinja2

_aimessage_template = jinja2.Template(    
    """<table>
    <thead><tr><td colspan="2">AI message</td></tr></thead>
    <tbody>
    <tr>
      {% if response.content %}
      <td style="vertical-align: top; border-right: 2px solid rgb(40, 40, 150); width: 6em;"><b>response:</b></td>
      <td style="text-align: left; text-indent: 0px;">
        <div style="white-space: pre-wrap;">
        {{ response.content }}
        </div>
      </td>
      {% endif %}
      {% if response.tool_calls %}
      <td style="vertical-align: top; border-right: 2px solid rgb(150, 40, 40); width: 6em;"><b>tools:</b></td>
      <td style="text-align: left; text-indent: 0px;">
        <div style="white-space: pre-wrap;">
        {{ response.tool_calls }}
        </div>
      </td>
      {% endif %}
  </tr></tbody></table>
  """)


def aimessage_html(obj):
    return _aimessage_template.render(response=obj)
    
html_formatter = get_ipython().display_formatter.formatters['text/html']
html_formatter.for_type_by_name('langchain_core.messages.ai', 'AIMessage', aimessage_html)

In [5]:
def toolmessage_html(obj):
    _toolmessage_template = jinja2.Template(    
        """<table>
        <thead><tr><td colspan="2">Tool message</td></tr></thead>
        <tbody>
        <tr>
          {% if response.content %}
          <td style="vertical-align: top; border-right: 2px solid rgb(40, 40, 150); width: 6em;"><b>response:</b></td>
          <td style="text-align: left; text-indent: 0px;">
            <div style="white-space: pre-wrap;">
            {{ response.content }}
            </div>
          </td>
          {% endif %}
        </tr>
        </tbody></table>
        """)
    return _toolmessage_template.render(response=obj)


html_formatter = get_ipython().display_formatter.formatters['text/html']
html_formatter.for_type_by_name('langchain_core.messages.tool', 'ToolMessage', toolmessage_html)

In [6]:
def highlight(text):
    return f'\n\033[1m{text}\033[0m'

## List of language models

Below is the list all models you want to use here. Be mindful about the capabilities of the system this will run on, in particular the RAM or the VRAM is a GPU is present.

This notebook was created with `gemma3:4b` (Gemma3 with 4B parameters) and `granite4:1b` (Granite4 with 1B parameters).

In [7]:
MODELS = (
    # OllamaModel('deepseek-r1:1.5b'),
    OllamaModel('gemma3:4b'),
    OllamaModel('functiongemma:270m'),
    OllamaModel('granite4:1b'),
    # OllamaModel('phi4-mini:3.8b'),
    # OllamaModel('qwen3:4b')
)

We are ready to start. First, we ensure that the models we need are pulled locally.

In [8]:
for mdl in MODELS:
    if mdl.is_pulled():
        print(f'Model {mdl} already pulled.')
    else:
        print(f'Pulling model {mdl}...', end='', flush=True)
        mdl.pull()
        print('done.')

Model gemma3:4b already pulled.
Pulling model functiongemma:270m...done.
Model granite4:1b already pulled.


# Language models and their limitations

We'll start a simple question that demonstrate that LLMs are eerily good at the pattern matching
part of understanding, but might not be able to go beyond that.

In [9]:
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.system import SystemMessage
from langchain_core.tools import tool

In [10]:
human_message = HumanMessage('Make the following calculation: 392373366237 * 562310951975')
for mdl in MODELS:
    print(f'\n\033[1m{str(mdl)}\033[0m\n')
    chat = mdl.new_chat()
    response = chat.invoke([human_message])
    display(response)


gemma3:4b



AIMessage(content="Okay, let's calculate 392373366237 * 562310951975.\n\nThis is a large multiplication, and it's best done using a calculator or a programming language.  Here's the result:\n\n**2236358838888888888888888888888888888888", additional_kwargs={}, response_metadata={}, id='lc_run--019b374f-6e3a-71f1-becf-cc95e394dbaa-0')


functiongemma:270m



AIMessage(content='I apologize, but I cannot perform this calculation. The calculation tool I have access to is specialized for calculating mathematical operations related to financial calculations, such as calculating stock market valuations or stock market indices. I cannot access or perform complex financial data like this.', additional_kwargs={}, response_metadata={'model': 'functiongemma:270m', 'created_at': '2025-12-19T15:52:32.893337884Z', 'done': True, 'done_reason': 'stop', 'total_duration': 15383443989, 'load_duration': 1883625294, 'prompt_eval_count': 42, 'prompt_eval_duration': 4166037781, 'eval_count': 51, 'eval_duration': 9261176107, 'logprobs': None, 'model_name': 'functiongemma:270m', 'model_provider': 'ollama'}, id='lc_run--019b374f-d562-7010-ab33-45343303aabf-0', usage_metadata={'input_tokens': 42, 'output_tokens': 51, 'total_tokens': 93})


granite4:1b



AIMessage(content='The result of multiplying 392373366237 by 562310951975 is:\n\n220,646,204,278,964,000,795', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-19T15:52:43.390581719Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10459409991, 'load_duration': 5442385739, 'prompt_eval_count': 46, 'prompt_eval_duration': 719383588, 'eval_count': 31, 'eval_duration': 4278871024, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b3750-11a1-7c22-be68-7bb258713622-0', usage_metadata={'input_tokens': 46, 'output_tokens': 31, 'total_tokens': 77})


Is this correct? The actual answer is:

In [11]:
print(f'{392373366237 * 562310951975:,}')

220,635,841,098,362,793,468,075


Unless this notebook is running in a future when LLMs have learned to multiply, or this notebook became part of the training data, the LLM answer is either likely not correct **OR** the LLM declines to provide an answer to the question (which is the case with `functiongemma`).

One should not rely on the current LLM technology to perform correctly mathematical operations, or more generally to evaluate expressions or run algorithms. However, LLMs have seen training data that describes steps
to resolve similar problems. They might be able to generate code (software instructions), for example a function call, to get to the answer. This can already be observed when using `gemma3:4b`: the answer from the LLM shows includes Python code that returns the correct answer is evaluated.

If we have a way to know when LLM answers contains code, we could simply evaluate that code and consider the resulting text its answer a dialogue. In fact, it does not even have to be a dialogue and code evaluation represent simply an action.

Additional instructions in the prompt can get us there with this example. We can add to it that if evaluating code seems better path to the correct answer, only code should be returned. To facilitate the triaging of answers (answer is text / the direct answer *vs* answer is code to get to the answer), we can also indicate the format to use.

In [12]:
answer_as_code = HumanMessage("""
IF this type of question would be better answered by evaluating code, return instead
ONLY Python code in the answer. Use the Markdown format for code snippets:
```python
<code>
```
""")
human_message = HumanMessage("""Make the following calculation: 392373366237 * 562310951975""")
for mdl in MODELS:
    print(highlight(str(mdl)))
    chat = mdl.new_chat()
    response = chat.invoke([human_message, answer_as_code])
    display(response)


gemma3:4b


AIMessage(content='```python\nprint(392373366237 * 562310951975)\n```', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-12-19T15:53:08.93582477Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5602291748, 'load_duration': 134981785, 'prompt_eval_count': 83, 'prompt_eval_duration': 993722234, 'eval_count': 35, 'eval_duration': 4406832911, 'logprobs': None, 'model_name': 'gemma3:4b', 'model_provider': 'ollama'}, id='lc_run--019b3750-8863-7290-80ad-d368f2a215e3-0', usage_metadata={'input_tokens': 83, 'output_tokens': 35, 'total_tokens': 118})


functiongemma:270m


AIMessage(content='I apologize, but I cannot assist with generating Python code. My current capabilities are limited to assisting with mathematical calculations and data analysis using the provided tools. I cannot generate programming code.', additional_kwargs={}, response_metadata={'model': 'functiongemma:270m', 'created_at': '2025-12-19T15:53:11.951815318Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3000338835, 'load_duration': 226857731, 'prompt_eval_count': 87, 'prompt_eval_duration': 119024520, 'eval_count': 37, 'eval_duration': 2595421552, 'logprobs': None, 'model_name': 'functiongemma:270m', 'model_provider': 'ollama'}, id='lc_run--019b3750-9e56-7ff3-b3dd-a8aceac0c091-0', usage_metadata={'input_tokens': 87, 'output_tokens': 37, 'total_tokens': 124})


granite4:1b


AIMessage(content='```python\nresult = 392373366237 * 562310951975\nresult\n```', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-19T15:53:17.572282746Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5601954745, 'load_duration': 57452067, 'prompt_eval_count': 86, 'prompt_eval_duration': 3056007529, 'eval_count': 21, 'eval_duration': 2472484344, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b3750-aa21-77a1-ad0b-a6962088e67f-0', usage_metadata={'input_tokens': 86, 'output_tokens': 21, 'total_tokens': 107})

This is looking promising, except for `functiongemma` that is adamant about being unabe to generate code.

A good check to perform with LLMs is whether a new prompt causes regression (that is loose the ability to answer some questions correctly). To achieve this we ask a question that should be answerable without code.
promptThat is a question for which the answer was almost certainly in the training data for the model.

In [13]:
human_message = HumanMessage("""What is the name of the natural satellite orbiting around the earth?""")
for mdl in MODELS:
    print(highlight(str(mdl)))
    chat = mdl.new_chat()
    response = chat.invoke([human_message, answer_as_code])
    display(response)


gemma3:4b


AIMessage(content='```python\nprint("The name of the natural satellite orbiting around the Earth is the Moon.")\n```', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-12-19T15:53:21.633500365Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4049036543, 'load_duration': 111993739, 'prompt_eval_count': 64, 'prompt_eval_duration': 1229417765, 'eval_count': 22, 'eval_duration': 2693744238, 'logprobs': None, 'model_name': 'gemma3:4b', 'model_provider': 'ollama'}, id='lc_run--019b3750-c00f-7c83-8f2f-e40518eec97d-0', usage_metadata={'input_tokens': 64, 'output_tokens': 22, 'total_tokens': 86})


functiongemma:270m


AIMessage(content='I apologize, but I cannot assist with programming or generating Python code. My current capabilities are limited to assisting with tasks related to natural language processing and text generation. I cannot provide programming advice or code snippets.', additional_kwargs={}, response_metadata={'model': 'functiongemma:270m', 'created_at': '2025-12-19T15:53:30.877379511Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9227157974, 'load_duration': 383140829, 'prompt_eval_count': 68, 'prompt_eval_duration': 3534807275, 'eval_count': 42, 'eval_duration': 5278100625, 'logprobs': None, 'model_name': 'functiongemma:270m', 'model_provider': 'ollama'}, id='lc_run--019b3750-cff1-7702-b363-1c2d78f4fc3b-0', usage_metadata={'input_tokens': 68, 'output_tokens': 42, 'total_tokens': 110})


granite4:1b


AIMessage(content='The name of the natural satellite orbiting around the Earth is called **the Moon**.', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-19T15:53:37.719574764Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6823422072, 'load_duration': 75365876, 'prompt_eval_count': 83, 'prompt_eval_duration': 4088770527, 'eval_count': 19, 'eval_duration': 2645278208, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b3750-f40f-7450-afb9-3ce197b6039d-0', usage_metadata={'input_tokens': 83, 'output_tokens': 19, 'total_tokens': 102})

While `gemma3:4b` its technically correct, it may indicate that this model will resort to writing code more often than necessary. `functiongemma` is consistent in messaging that it cannot generate code.

# Adding tools as context

Agents are language models to which the availability of tools (software functions), with a description of what they do, is specified as context. For `gemma3:4b` this can be achieved through a system prompt, while a few other models with a "tools" capability Python functions can passed directly. This is the case with `granite4:1b` for example.

## Tools in prompt

We showed earlier that a prompt can indicate that code evaluation is available.
Similarly, we can indicate that tools are available and calling a tool can be used as an answer.

If we make our multiplication a tool, a prompt can look like follows:

In [14]:
system_prompt_tools = """
You have access to functions. 
You don't have to use tools if not necessary, but *IF* you decide to invoke any of the function(s),
you MUST put it in a valid JSON expression like:
[
  {"name": function name,
   "parameters": dictionary of argument name and its value}
]

You SHOULD NOT include any other text in the response if you call a function.

Functions:
[
  { "name": "multiply",
    "description": "Multiply two numbers.",
    "parameters": {
      "type": "object",
      "properties": {
          "a": "number",
          "b": "number"
      },
      "required": [
          "a", "b"
      ]
    }
  }
]
"""

## Tools as `tools`

The use of tools has been somewhat formalized since the early days of prompt engineering, a mere double-digit number
of months in the past, we have seen so far.

Many of the most recent language model specifically include a capability called "tools". This frees a user, or developer,
from having to work on prompt sections that declare the availablity of tools in a way that the language model reacts to it, and on parsing out if an answer is text or code. `langchain` provides a wrapper for this and let one simply declare any Python function as tool. For the multiplication we can just plug the Python multiplication operator.

In [15]:
import operator
tools = [tool(operator.mul)]

## Language model opting to answer with a tool call

We have shown that a prompt context, wrapped in a `langchain` tool declaration when the model has the capability, can lead a language model to answer with code. If we ask again our multiplication question:

In [16]:
human_message = HumanMessage("""Make the following calculation: 392373366237 * 562310951975""")

for model in MODELS:
    chat = model.new_chat()
    if model.has_tools():
        print(f'{highlight(model.name)} (with tools)\n')
        chat = chat.bind_tools(tools)
        messages = [human_message]
    else:
        print(f'{highlight(model.name)}\n')
        messages = [SystemMessage(system_prompt_tools),
                    human_message]
    response = chat.invoke(messages)
    display(response)


gemma3:4b



AIMessage(content='```json\n[\n  {\n    "name": "multiply",\n    "parameters": {\n      "a": 392373366237,\n      "b": 562310951975\n    }\n  }\n]\n```', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-12-19T15:56:17.201230892Z', 'done': True, 'done_reason': 'stop', 'total_duration': 17443248812, 'load_duration': 154185821, 'prompt_eval_count': 224, 'prompt_eval_duration': 4023212770, 'eval_count': 69, 'eval_duration': 13217112473, 'logprobs': None, 'model_name': 'gemma3:4b', 'model_provider': 'ollama'}, id='lc_run--019b3753-398c-7a53-bc6f-8f7f040cd29f-0', usage_metadata={'input_tokens': 224, 'output_tokens': 69, 'total_tokens': 293})


functiongemma:270m (with tools)



AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'functiongemma:270m', 'created_at': '2025-12-19T15:56:20.54147123Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3211586801, 'load_duration': 267494815, 'prompt_eval_count': 126, 'prompt_eval_duration': 242371681, 'eval_count': 37, 'eval_duration': 2679888246, 'logprobs': None, 'model_name': 'functiongemma:270m', 'model_provider': 'ollama'}, id='lc_run--019b3753-7e30-7b30-8ce7-165750feb86e-0', tool_calls=[{'name': 'mul', 'args': {'a': 392373366237, 'b': 562310951975}, 'id': '7c9769d1-8531-4868-866f-3d00eb598097', 'type': 'tool_call'}], usage_metadata={'input_tokens': 126, 'output_tokens': 37, 'total_tokens': 163})


granite4:1b (with tools)



AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-19T15:56:30.374992769Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9785316390, 'load_duration': 82755867, 'prompt_eval_count': 199, 'prompt_eval_duration': 6080778373, 'eval_count': 31, 'eval_duration': 3601300486, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b3753-8aec-77b2-a195-77f02da8f5d8-0', tool_calls=[{'name': 'mul', 'args': {'a': '392373366237', 'b': '562310951975'}, 'id': '62be38a6-3878-4ad3-839d-31093d5d8d3e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 199, 'output_tokens': 31, 'total_tokens': 230})

All language models understand the assignment and should provide correct function calls. Interestingly, creating a function call is creating code which `functiongemma` is otherwise convinced it is unable to do.

# Agentic linear chaining - an example

## Database

We create a local database for demonstration purposes.

In [17]:
import sqlite3

dbcon = sqlite3.connect(':memory:', check_same_thread=False)
dbcon.execute("""
CREATE TABLE medication(
ndc STRING PRIMARY KEY,
common_name STRING,
quantity INTEGER
);
""")
for values in (
    ('ABCD-DEF', 'Aspirin', 3), ('ABCD-GHI', 'Aspirin', 5), ('GHIK-KLM', 'bacitracin', 10)
):
    dbcon.execute("""
    INSERT INTO medication(ndc, common_name, quantity) VALUES (?,?,?);
    """, values)



## Tools to query a database

Functions to query the database:
- `get_ndc_code` to get all NDC codes for the common name of a medication
- `get_medication_stock` to get the stock for a given NDC code

In [18]:
def get_ndc_code(medication: str) -> tuple[str, ...]:
    """Get possible ndc codes for a medication.
    
    Args:
      medication: name of the medication for which NDC code should be returned.
    """
    cursor = dbcon.cursor()
    cursor.execute('SELECT ndc FROM medication WHERE common_name==?',
                   (medication, ))
    return tuple(row[0] for row in cursor.fetchall())


def get_ndc_stock(ndc_code: str) -> int:
    """Get the number of boxes of medication in stock.
    
    Args:
      ndc_code: NDC code to retrieve stock for.
    """
    cursor = dbcon.cursor()
    cursor.execute('SELECT quantity FROM medication WHERE ndc==?',
                   (ndc_code, ))
    return cursor.fetchone()[0]

## Prompting an LLM to use tools (when needed)

The prompt may contain several tools available. A prompt context will be similar to the one we showed for the multiplication. 

In [19]:
system_prompt_tools = """
You have access to functions. 
You don't have to use tools if not necessary, but *IF* you decide to invoke any of the function(s),
you MUST put it in a valid JSON expression like:
[
  {"name": function name,
   "parameters": dictionary of argument name and its value}
]

You SHOULD NOT include any other text in the response if you call a function.

Functions:
[
  {
    "name": "get_ndc_stock",
    "description": "Get the number of boxes of medication in stock.",
    "parameters": {
      "type": "object",
      "properties": {
        "ndc_code": {
          "type": "string"
        }
      },
      "required": [
        "ndc_code"
      ]
    }
  },
  {
    "name": "get_ndc_code",
    "description": "Get possible ndc codes for a medication.",
    "parameters": {
      "type": "object",
      "properties": {
        "medication": {
          "type": "string"
        }
      },
      "required": [
        "medication"
      ]
    },
  }
]
"""

Whenever a model has the capabilities "tools", the user or developer is spared the prompt context engineering.
The decorator `tool()` in `langchain` can be used instead.

Note: `langchain` is essentially an abstraction layer over prompts.
It can create the equivalent of our engineered prompt contexts from Python function definitions,
feeding the LLM with information in the docstring, or in additional declarative structures
(see https://docs.langchain.com/oss/python/langchain/tools#advanced-schema-definition).

In [20]:
tools = [tool(get_ndc_code), tool(get_ndc_stock)]

In [21]:
def invoke_model(model, human_message, system_message=SystemMessage('')):
    chat = model.new_chat()
    if model.has_tools():
        print(f'\n{highlight(model.name)} (with tools)\n')
        chat = chat.bind_tools(tools)
    else:
        print(f'\n{highlight(model.name)}\n')
    messages = [system_message,
                human_message]
    response = chat.invoke(messages)
    return response, chat

human_message = HumanMessage('Name the first month of the year.')

for model in MODELS:
    response, chat = invoke_model(model, human_message)
    display(response)



gemma3:4b



AIMessage(content='January! \n\nDo you want to play a quick game of guessing months? 😊', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-12-19T15:56:44.035293361Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3309531741, 'load_duration': 157166245, 'prompt_eval_count': 22, 'prompt_eval_duration': 474170429, 'eval_count': 18, 'eval_duration': 2671110580, 'logprobs': None, 'model_name': 'gemma3:4b', 'model_provider': 'ollama'}, id='lc_run--019b3753-d994-75d0-95c4-ac1ee09da8c9-0', usage_metadata={'input_tokens': 22, 'output_tokens': 18, 'total_tokens': 40})



functiongemma:270m (with tools)



AIMessage(content='I apologize, but I cannot assist with retrieving the specific NDC code for the first month of the year. My current tools are designed for retrieving medication-related data and stock information. I cannot query or retrieve specific calendar or stock data for specific dates.', additional_kwargs={}, response_metadata={'model': 'functiongemma:270m', 'created_at': '2025-12-19T15:56:56.763608842Z', 'done': True, 'done_reason': 'stop', 'total_duration': 12611970214, 'load_duration': 288091059, 'prompt_eval_count': 187, 'prompt_eval_duration': 4427416337, 'eval_count': 51, 'eval_duration': 7846284522, 'logprobs': None, 'model_name': 'functiongemma:270m', 'model_provider': 'ollama'}, id='lc_run--019b3753-e6f6-70a1-bffd-6fc74374d490-0', usage_metadata={'input_tokens': 187, 'output_tokens': 51, 'total_tokens': 238})



granite4:1b (with tools)



AIMessage(content='The first month of the year is **January**.', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-19T15:57:04.964227029Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8139267110, 'load_duration': 67773414, 'prompt_eval_count': 268, 'prompt_eval_duration': 6357916317, 'eval_count': 12, 'eval_duration': 1695143557, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b3754-1877-7431-a665-d9602c0eeaaa-0', usage_metadata={'input_tokens': 268, 'output_tokens': 12, 'total_tokens': 280})

`functiongemma:270` appears challenged with the question. Its size (just over 1/4th of `granite4:1b`) might be the issue.

In [22]:
tool_map = {tool.name: tool for tool in tools}
if len(tool_map) != len(tools):
    raise ValueError("'tools' contains at lease one tool name duplicate.")

In [23]:
human_message = HumanMessage('How many boxes of medication with NDC code ABCD-DEF do we have left?')

for model in MODELS:
    response, chat = invoke_model(model, human_message, SystemMessage(system_prompt_tools))
    display(response)
    if response.tool_calls:
        for tool_call in response.tool_calls:
            print(
                f'  --> calling {tool_call["name"]}'
                f'({", ".join("=".join((k, repr(v))) for k, v in tool_call["args"].items())})'
            )
            selected_tool = tool_map[tool_call['name']]
            res = selected_tool.invoke(tool_call)
            print('      result:')
            print(f'        {res.content}')
    else:
        print('Assessing automatically if there is a tool call is left as an exercise for the reader.')



gemma3:4b



AIMessage(content='```json\n[\n  {\n    "name": "get_ndc_stock",\n    "parameters": {\n      "ndc_code": "ABCD-DEF"\n    }\n  }\n]\n```', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-12-19T15:57:22.561638059Z', 'done': True, 'done_reason': 'stop', 'total_duration': 17479447897, 'load_duration': 171918526, 'prompt_eval_count': 318, 'prompt_eval_duration': 5923119520, 'eval_count': 50, 'eval_duration': 11354512648, 'logprobs': None, 'model_name': 'gemma3:4b', 'model_provider': 'ollama'}, id='lc_run--019b3754-38b9-79c2-bf49-feebe189954e-0', usage_metadata={'input_tokens': 318, 'output_tokens': 50, 'total_tokens': 368})

Assessing automatically if there is a tool call is left as an exercise for the reader.


functiongemma:270m (with tools)



AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'functiongemma:270m', 'created_at': '2025-12-19T15:57:30.780728734Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8043054729, 'load_duration': 245270405, 'prompt_eval_count': 483, 'prompt_eval_duration': 6881853573, 'eval_count': 23, 'eval_duration': 903915132, 'logprobs': None, 'model_name': 'functiongemma:270m', 'model_provider': 'ollama'}, id='lc_run--019b3754-7db0-7070-85c9-03324e9cf855-0', tool_calls=[{'name': 'get_ndc_stock', 'args': {'ndc_code': 'ABCD-DEF'}, 'id': 'a63df468-8c35-4d1c-ba01-f07c58b4a29a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 483, 'output_tokens': 23, 'total_tokens': 506})

  --> calling get_ndc_stock(ndc_code='ABCD-DEF')
      result:
        3


granite4:1b (with tools)



AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-19T15:57:50.134461636Z', 'done': True, 'done_reason': 'stop', 'total_duration': 19305905438, 'load_duration': 53794011, 'prompt_eval_count': 518, 'prompt_eval_duration': 14909120159, 'eval_count': 27, 'eval_duration': 4311319780, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b3754-9d4b-7ee0-a582-f89a70cdfa89-0', tool_calls=[{'name': 'get_ndc_stock', 'args': {'ndc_code': 'ABCD-DEF'}, 'id': 'fe96a455-02c4-4f10-ab45-e64a4d3ff059', 'type': 'tool_call'}], usage_metadata={'input_tokens': 518, 'output_tokens': 27, 'total_tokens': 545})

  --> calling get_ndc_stock(ndc_code='ABCD-DEF')
      result:
        3


In [24]:
human_message = HumanMessage("What are possible NDC codes for the medication 'Aspirin'?")

for model in MODELS:
    response, chat = invoke_model(model, human_message, SystemMessage(system_prompt_tools))
    display(response)
    if response.tool_calls:
        print('      result:')
        for tool_call in response.tool_calls:
            selected_tool = tool_map[tool_call['name']]
            res = selected_tool.invoke(tool_call)    
            print(f'        {res.content}')
    else:
        print('Assessing automatically if there is a tool call is left as an exercise for the reader.')



gemma3:4b



AIMessage(content='```json\n[\n  {\n    "name": "get_ndc_code",\n    "parameters": {\n      "medication": "Aspirin"\n    }\n  }\n]\n```', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-12-19T15:58:00.993286466Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10735424984, 'load_duration': 187402978, 'prompt_eval_count': 315, 'prompt_eval_duration': 952424576, 'eval_count': 48, 'eval_duration': 9541583935, 'logprobs': None, 'model_name': 'gemma3:4b', 'model_provider': 'ollama'}, id='lc_run--019b3754-e930-79c1-aec7-596a87b23c76-0', usage_metadata={'input_tokens': 315, 'output_tokens': 48, 'total_tokens': 363})

Assessing automatically if there is a tool call is left as an exercise for the reader.


functiongemma:270m (with tools)



AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'functiongemma:270m', 'created_at': '2025-12-19T15:58:06.693394285Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5520482910, 'load_duration': 284049169, 'prompt_eval_count': 480, 'prompt_eval_duration': 4023532210, 'eval_count': 21, 'eval_duration': 1176127051, 'logprobs': None, 'model_name': 'functiongemma:270m', 'model_provider': 'ollama'}, id='lc_run--019b3755-13d3-7130-9404-230265a62b9e-0', tool_calls=[{'name': 'get_ndc_code', 'args': {'medication': 'Aspirin'}, 'id': '5d987bf2-cc49-4058-b0d7-08196877a7d3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 480, 'output_tokens': 21, 'total_tokens': 501})

      result:
        ["ABCD-DEF", "ABCD-GHI"]


granite4:1b (with tools)



AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-19T15:58:11.403304958Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4647825876, 'load_duration': 48513311, 'prompt_eval_count': 514, 'prompt_eval_duration': 624487429, 'eval_count': 25, 'eval_duration': 3947049016, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b3755-29a2-7be1-b164-a298ea42b847-0', tool_calls=[{'name': 'get_ndc_code', 'args': {'medication': 'Aspirin'}, 'id': '65211877-5146-4db6-acfd-6c658edb630b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 514, 'output_tokens': 25, 'total_tokens': 539})

      result:
        ["ABCD-DEF", "ABCD-GHI"]


Cool. But what did we exactly gain here? If you are are programmer or a data analyst knowing how to write SQL, arguably not much. However, what we have achieved is akin to a text user interface, or language user interface, that can be an alternative to a graphical user interface.

## Reasoning in steps

So far we only showed questions for which either the language model had the answer memorized from the data it was trained on, or for which the answer was one tool call away.

There are questions questions for which the answer requires a sequence of recall from training data or tool calls, with the output of a step used as the input to the next step. For example, asking for how many Aspirin boxes are in stock, would require to 1) get the NDC codes for Aspirin products, 2) query the stock for each, and finally 3) sum those values.

In [25]:
human_message = HumanMessage('How many boxes of Aspirin do we have in stock altogether?')

for model in MODELS:
    response, chat = invoke_model(model, human_message, SystemMessage(system_prompt_tools))
    display(response)



gemma3:4b



AIMessage(content='```json\n[\n  {\n    "name": "get_ndc_stock",\n    "parameters": {\n      "ndc_code": "98304-600"\n    }\n  }\n]\n```', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-12-19T15:58:36.459900933Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9145060624, 'load_duration': 161680650, 'prompt_eval_count': 315, 'prompt_eval_duration': 503586310, 'eval_count': 56, 'eval_duration': 8434434702, 'logprobs': None, 'model_name': 'gemma3:4b', 'model_provider': 'ollama'}, id='lc_run--019b3755-79f1-7ed1-b621-80a21c777557-0', usage_metadata={'input_tokens': 315, 'output_tokens': 56, 'total_tokens': 371})



functiongemma:270m (with tools)



AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'functiongemma:270m', 'created_at': '2025-12-19T15:58:38.843606411Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2151825106, 'load_duration': 387322588, 'prompt_eval_count': 480, 'prompt_eval_duration': 243721144, 'eval_count': 23, 'eval_duration': 1501447963, 'logprobs': None, 'model_name': 'functiongemma:270m', 'model_provider': 'ollama'}, id='lc_run--019b3755-9e8b-7672-b0f7-f907ff14bc50-0', tool_calls=[{'name': 'get_ndc_stock', 'args': {'ndc_code': 'Aspirin'}, 'id': '7d0683fa-c668-4640-86c8-2b74bc4f273f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 480, 'output_tokens': 23, 'total_tokens': 503})



granite4:1b (with tools)



AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-19T15:58:46.687847406Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7795167505, 'load_duration': 52036147, 'prompt_eval_count': 514, 'prompt_eval_duration': 4347833691, 'eval_count': 25, 'eval_duration': 3366442035, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b3755-a72b-7521-849a-744947dfc784-0', tool_calls=[{'name': 'get_ndc_code', 'args': {'medication': 'Aspirin'}, 'id': 'bf98a6fe-bc7f-4027-ae38-1a8e8a9b8c2e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 514, 'output_tokens': 25, 'total_tokens': 539})

Odds are are that the results are not great. Bare language models used in this notebook generally do not handle this well without guidance from a prompt. For example,  

## ReAct

The ReAct approach prompts a language model to match its answers to the following pattern:
1. **question**
2. **thought:** the language model's answer to the question. The answer can be the use of a tool.
3. **action:** whenever the answer is the use of a tool, a function call, this is selected here.
4. **action input:** the input to the action.
5. **observation:** observe the result of performing the action.
6. The sequence of steps 2 to 5 can be repeated N times
7. **final answer**

It is derived from the empirical observation that LLMs appeared to give more "thoughtful" answers, or better solve reasoning-oriented questions when asked to think step by step.

React can be idependently achieved through adding the description to a prompt and iteratively parsing language model responses and crafting new messages. However, in many scenario this is can be abstracted out through the use of library. We use `langchain` again to demonstrate how it can work:

In [29]:
from langchain.agents import create_agent

human_message = HumanMessage("How many boxes of Aspirin do we have in stock altogether?")

for model in MODELS:
    print(highlight(model.name))
    if model.has_tools():
        react = create_agent(model.new_chat(), tools=tools)
        r_stream = react.stream({'messages': [human_message]}, stream_mode='updates')
        try:
            for chunk in r_stream:
                for k in chunk.keys():
                    for m in chunk[k]['messages']:
                        display(m)
        except TypeError as err:
            print('Error:')
            print(err)
    else:
        print('Parsing AI messages into next actions and results is left as an exercise.')


gemma3:4b
Parsing AI messages into next actions and results is left as an exercise.

functiongemma:270m


AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'functiongemma:270m', 'created_at': '2025-12-19T16:15:15.100583755Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1757541174, 'load_duration': 139784081, 'prompt_eval_count': 192, 'prompt_eval_duration': 6755575, 'eval_count': 23, 'eval_duration': 1597907890, 'logprobs': None, 'model_name': 'functiongemma:270m', 'model_provider': 'ollama'}, id='lc_run--019b3764-d3bd-7780-a4dc-f45106de7676-0', tool_calls=[{'name': 'get_ndc_stock', 'args': {'ndc_code': 'Aspirin'}, 'id': '0d55848c-3003-408c-846e-2ccb2238bc38', 'type': 'tool_call'}], usage_metadata={'input_tokens': 192, 'output_tokens': 23, 'total_tokens': 215})

Error:
'NoneType' object is not subscriptable

granite4:1b


AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-19T16:15:25.468328729Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10289600373, 'load_duration': 3945861777, 'prompt_eval_count': 274, 'prompt_eval_duration': 3503917652, 'eval_count': 25, 'eval_duration': 2819616871, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b3764-dae9-7263-bf54-e82336f4d328-0', tool_calls=[{'name': 'get_ndc_code', 'args': {'medication': 'Aspirin'}, 'id': '89afdd0e-92bf-4d6e-9270-10ec8dfb20c9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 274, 'output_tokens': 25, 'total_tokens': 299})

ToolMessage(content='["ABCD-DEF", "ABCD-GHI"]', name='get_ndc_code', id='d8cc0dd6-bb1f-4322-a0f0-911b83bc8221', tool_call_id='89afdd0e-92bf-4d6e-9270-10ec8dfb20c9')

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-19T16:15:29.79522246Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4318452056, 'load_duration': 53368600, 'prompt_eval_count': 323, 'prompt_eval_duration': 669659728, 'eval_count': 27, 'eval_duration': 3576577944, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b3765-0323-7931-9608-2cc0f0d1aaab-0', tool_calls=[{'name': 'get_ndc_stock', 'args': {'ndc_code': 'ABCD-DEF'}, 'id': '6886023e-3e2e-4ea3-a0f4-5e0a4e1b0c6a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 323, 'output_tokens': 27, 'total_tokens': 350})

ToolMessage(content='3', name='get_ndc_stock', id='63d75cad-c063-4d81-9e2e-294e0656cd99', tool_call_id='6886023e-3e2e-4ea3-a0f4-5e0a4e1b0c6a')

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-19T16:15:33.78495174Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3981372369, 'load_duration': 30995291, 'prompt_eval_count': 364, 'prompt_eval_duration': 365822022, 'eval_count': 26, 'eval_duration': 3565190022, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b3765-140a-74f1-ab9c-4c5dbd4fc052-0', tool_calls=[{'name': 'get_ndc_stock', 'args': {'ndc_code': 'ABCD-GHI'}, 'id': '7e28b8c2-e435-4474-86ef-e55bc82bc785', 'type': 'tool_call'}], usage_metadata={'input_tokens': 364, 'output_tokens': 26, 'total_tokens': 390})

ToolMessage(content='5', name='get_ndc_stock', id='99690172-b124-451b-832f-3432de7a99fe', tool_call_id='7e28b8c2-e435-4474-86ef-e55bc82bc785')

AIMessage(content='The total number of boxes in stock for Aspirin is **8**.', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-19T16:15:36.729298734Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2936094184, 'load_duration': 49801926, 'prompt_eval_count': 405, 'prompt_eval_duration': 267335280, 'eval_count': 17, 'eval_duration': 2603902041, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b3765-23a0-7241-aa3e-7453857123e2-0', usage_metadata={'input_tokens': 405, 'output_tokens': 17, 'total_tokens': 422})

When calling tools works without error, the answer is indeed correct.

# Conclusion

Small language models are able to handle simple queries and chain sequences of steps to reach an answer. They can run locally / isolated and on relatively modest hardware.